Neural networks learn intermediate representations of the data, typically in a different-dimensional space.

Torch is the tensor backend under PyTorch (an alternative to, say, Theano).

PyTorch tensors can benefit from very fast computations on GPUs. **What about TPUs?** They also keep track of the operations that created them, which is used by autodiff.

In [6]:
import torch

a = [1.0, 2.0, 3.0]

torch.ones(size=(3,2))

tensor([1., 1., 1., 1., 1., 1.])

In [31]:
(
    torch.arange(start=1, end=10, step=1, dtype=torch.int)
    .reshape((3,3))
)


tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]], dtype=torch.int32)

In [33]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [36]:
points.shape

torch.Size([3, 2])

In [37]:
torch.zeros((3,2))

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [44]:
points[1,1]

tensor(3.)

In [49]:
points[:,0]

tensor([4., 5., 2.])

The above is a *view* on the underlying data, 

In [52]:
points[None]  # adds a dimension!

tensor([[[4., 1.],
         [5., 3.],
         [2., 1.]]])

In [68]:
# also adds a dimension!
# dim arg is which dimension to insert
# torch.unsqueeze?

points.unsqueeze(0)

tensor([[[4., 1.],
         [5., 3.],
         [2., 1.]]])

In [77]:
print(points.unsqueeze(-1).shape)
points.unsqueeze(-1)[0,0,0]

torch.Size([3, 2, 1])


tensor(4.)

In [85]:
# single grayscale image
img_t = torch.randn((3, 5, 5))  # [channels, rows, columns]
weights = torch.tensor([0.2126, 0.7152, 0.0722])


# batch of 2 grayscale images
batch_t = torch.randn((2, 3, 5, 5))  # [image, channels, rows, columns]

In [96]:
print(f"batch_t.shape:    {batch_t.shape}")
print(f"batch_t[0].shape: {batch_t[0].shape}")

batch_t.shape:    torch.Size([2, 3, 5, 5])
batch_t[0].shape: torch.Size([3, 5, 5])


In [107]:
img_mean = img_t.mean(dim=0)  # average over the channel dimension
print(img_mean.shape)

torch.Size([5, 5])


In [108]:
batch_means = batch_t.mean(dim=-3)  # average over the channel dimension

In [129]:
# unsqueeze weights to make them the same size as img
print(weights.unsqueeze(-1).unsqueeze(-1).shape)
print(img_t.shape)
img_weighted = (img_t * weights.unsqueeze(-1).unsqueeze(-1))
print(img_weighted.shape)
img_weighted = img_weighted.sum(dim=0)
print(img_weighted.shape)

print('\n')
print(batch_t.shape)
batch_weighted = (batch_t * weights.unsqueeze(-1).unsqueeze(-1))
print(batch_weighted.shape)
batch_weighted = batch_weighted.sum(dim=-3)
print(batch_weighted.shape)

torch.Size([3, 1, 1])
torch.Size([3, 5, 5])
torch.Size([3, 5, 5])
torch.Size([5, 5])


torch.Size([2, 3, 5, 5])
torch.Size([2, 3, 5, 5])
torch.Size([2, 5, 5])


Torch has named tensors which help reduce confusion! This is technically still an "experimental feature," though.

In [132]:
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])
weights_named

tensor([0.2126, 0.7152, 0.0722], names=('channels',))

In [146]:
img_named = img_t.refine_names(..., 'channels', 'rows', 'columns')
print(img_named.names)


('channels', 'rows', 'columns')


In [152]:
# weights_named.align_as?
# useful for aligning two tensors' dimensions
weights_aligned = weights_named.align_as(other=img_named)

In [157]:
print(weights_aligned.shape)
print(img_named.shape)


torch.Size([3, 1, 1])
torch.Size([3, 5, 5])


torch.Size([3, 5, 5])

Some functions also take names for dimensions!

In [158]:
img_named.sum(dim='channels')

tensor([[-1.4610, -1.6321, -0.8742, -0.2011,  4.5843],
        [ 0.6535, -2.2531, -1.3374,  1.7136,  0.7354],
        [-0.5603, -0.7490,  2.1725, -1.6518,  1.3674],
        [-1.7368, -0.2673, -1.0038, -1.4500,  2.5406],
        [ 0.1897,  0.1922, -1.8367, -1.9601, -0.2229]],
       names=('rows', 'columns'))

We can also un-name them, to avoid issues:

In [160]:
weights_named.rename(None)

tensor([0.2126, 0.7152, 0.0722])